<a href="https://colab.research.google.com/github/szyxxx/MachineLearning-TelkomUniversity/blob/main/Modern-CNN/NiN/AxelDavid_1103210017_TK4504_NiN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Axel David<br>
NIM: 1103210017<br>
Judul Tugas: Pembuatan Model CNN Modern - NiN<br>
Lecture: 13<br>
Model: NiN<br>
Dataset: CIFAR-10

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import tensorflow as tf
from PIL import Image

In [17]:
# Mengunduh dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalisasi dataset
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Konversi label ke tensor
train_labels = torch.tensor(train_labels, dtype=torch.long).squeeze()
test_labels = torch.tensor(test_labels, dtype=torch.long).squeeze()

# Membuat transformasi
transform = transforms.Compose([
    transforms.ToPILImage(),  # Mengonversi gambar menjadi PIL Image
    transforms.Resize(224),  # Mengubah ukuran gambar menjadi 224x224
    transforms.ToTensor(),  # Mengonversi gambar menjadi tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalisasi gambar
])

# Membuat dataset CIFAR-10
class CIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

trainset = CIFAR10Dataset(train_images, train_labels, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = CIFAR10Dataset(test_images, test_labels, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

In [18]:
# Mendefinisikan arsitektur NiN (Network in Network)
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU()
    )

class NiN(nn.Module):
    def __init__(self):
        super(NiN, self).__init__()
        self.net = nn.Sequential(
            nin_block(3, 96, kernel_size=11, stride=4, padding=0),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nin_block(96, 256, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nin_block(256, 384, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(0.5),
            nin_block(384, 10, kernel_size=3, stride=1, padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten()
        )

    def forward(self, x):
        return self.net(x)

net_nin = NiN()

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net_nin.parameters(), lr=0.01, momentum=0.9)

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_nin.to(device)

num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net_nin(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Cetak setiap 10 batch
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10:.3f}")
            running_loss = 0.0

print("Finished Training")


[1, 200] loss: 46.068
[2, 200] loss: 46.052
[3, 200] loss: 46.052
[4, 200] loss: 46.052
[5, 200] loss: 46.052
Finished Training


In [21]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net_nin(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 10.06%


In [22]:
torch.save(net_nin.state_dict(), 'nin_cifar10.pth')